<img src="https://lh3.googleusercontent.com/pw/ACtC-3fFHZrzKpHGWl0vYz7Sr8FX8QqLQ_tc8XHBSwqQnM4hgsIOjtjaOde1M9oHSAfe1Fs2SwVORlapit4-JOz0mjP8Tnz6HetkLZDZb8CifSd0uoSp1Nj3wG_wh1sEQlKXXzvEA9Y9HnQqu2Ecv2igmInb=w1097-h235-no?authuser=0" alt="2020年度ゲノム情報解析入門" height="100px" align="middle">

# 機械学習 - 線形回帰（2）-

　「[機械学習 - 線形回帰（1）-]((https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2020/L12_ML_regression_1.ipynb))」では、説明変数（1遺伝子の発現量）をひとつ使い、その遺伝子発現量から表現型値を予測する線形回帰モデルを構築しました。しかし、そのモデルの評価値（決定係数 $R^2$ ）は低く、良いモデルではありませんでした。

　今回、使用する説明変数を増やして、より良い線形回帰モデルを作ってみましょう。

---

## 今回の実習内容

1. 線形回帰 Linear regression（2変数以上）


### 線形回帰モデル


　ここでおこなう解析の目標は次のとおりです。
> 遺伝子発現量から表現型値を予測する線形回帰モデルを作る

　「線形回帰モデル」とは何か？　遺伝子発現量（$x$）と表現型値（$y$）の関係を次の方程式で表したモデルのことです。

線形回帰モデルの方程式: 

<img src="https://github.com/CropEvol/lecture/blob/2018/textbook_2018/09_statistics/data/regression_base.png?raw=true" alt="regression" height="130px">

- **目的変数（objective variable）**: 予測される変数$y$。今回の場合、表現型値。
- **説明変数（explanatory variable）**: 予測に使う変数$x$。今回の場合、各遺伝子発現量。
- **偏回帰係数（coefficient）**: 各説明変数の重み。目的変数の予測にその変数がどれぐらい影響するかを示す指標。
- **誤差（intercept; 切片）**: 説明変数以外の影響を示す項。

　偏回帰係数や誤差の値は、データから推定します。「線形回帰モデルを作る」とは、それらの推定値を得ることに相当します。

### 実習で使用するデータセット

　次のコードセルを実行して、データファイル（[gene_expression.csv](https://github.com/CropEvol/lecture/blob/master/textbook_2019/dataset/gene_expression.csv)）をダウンロードしてください。

ファイルの詳細:
- ファイル名: gene_expression.csv
- カンマ区切りテキストファイル
- 100行（100サンプル） x 51列（表現型値 + 50個の遺伝子発現量）


In [ ]:
### このコードセルは実行のみ ###
# サンプルデータのダウンロード
!wget -q -O gene_expression.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2019/dataset/gene_expression.csv

# pandasで読み込み
import pandas as pd
df = pd.read_csv("gene_expression.csv", sep=",", header=0)
df

## 2. 線形回帰 Linear regression（2変数以上）

　使用する説明変数を3つに増やして、少し良いモデルを作ってみましょう。

使用する説明変数
- gene_7の遺伝子発現量 `gene_7`
- gene_11の遺伝子発現量 `gene_11`
- gene_28の遺伝子発現量 `gene_28`

> 3個の遺伝子発現量から表現型値（phenotype）を予測する線形回帰モデルを作る
  $$ y = \beta_{gene\_7} x_{gene\_7} + \beta_{gene\_11} x_{gene\_11} + \beta_{gene\_28} x_{gene\_28} + e $$

　説明変数の個数が2つ以上になっても、基本的な手順は1変数のときと同じです。異なる点は、前処理に**スケーリング（scaling）**と呼ばれる操作が入る点です。

- 2-1. 前処理
  - **スケーリング（scaling）**
- 2-2. モデルの選択
- 2-3. モデルの学習
- 2-4. モデルの評価
- 2-5. 予測

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/supervised_learning_process.png?raw=true" alt="supervised_learning_process" height="60px">


### 2-1. 前処理

In [ ]:
import numpy as np

# 使用する変数
x = np.array(df.loc[:,["gene_7", "gene_11", "gene_28"]]) # 説明変数3つ
y = np.array(df["phenotype"])                             # 目的変数

# データ分割: 100サンプルを「トレーニングデータ70%、テストデータ30%」に分ける
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

print("training: ", x_train.shape, y_train.shape)
print("test: ", x_test.shape, y_test.shape)


スケーリングにはおもに2つの方法があります。 
- **正規化 normalization**: 各変数の値を0 ~ 1の範囲に変換する  
```python
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()  # 変数名「mms」の部分は任意
x_train_mms = mms.fit_transform(x_train)
x_test_mms = mms.transform(x_test)
```

- **標準化 standardization**: 各変数の値を平均値0、標準偏差1になるように変換する
```python
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() # 変数名「ss」の部分は任意
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.transform(x_test)
```

　ここでは、標準化をおこないます。

<small>※ 詳細は省きますが、複数の説明変数（身長や体重など）の単位を揃えずにモデルの学習をおこなうと、学習効率や予測モデルの悪影響を及ぼします。また、得られた学習結果を解釈する（影響力の大きい説明変数がどれかを判断する）ためにも重要な操作です。</small>

In [ ]:
# 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.transform(x_test)

# 確認
#x_train_ss
x_test_ss

### 2-2 ~ 2-3. モデルの選択・学習

　線形回帰モデルを用意し、スケーリング後のトレーニングデータでモデルの学習をおこないます。

In [ ]:
# モデル選択＆学習
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_train_ss, y_train)

# 係数b、誤差e
b = lr.coef_
e = lr.intercept_
print("Coefficient=", b)
print("Intercept="  , e)

### 各項の係数 Coefficient について

　$$ 線形回帰モデル: f(x) = \beta_1x_1 + \beta_2x_2 + \beta_3x_3 + ... + \beta_mx_m+ e $$

　各項の係数$\beta$（Coefficient）は、それぞれの説明変数$x_1, x_2...x_m$が目的変数$y$にどの程度影響を持っているかを表しています。影響の大きさを知りたいのであれば、係数の絶対値をみます。影響の方向性（プラスかマイナスか）を知りたいのであれば、係数の符号をみます。


　詳細は省きますが、前処理でおこなったスケーリングはこの係数に影響してきます。モデルの各係数の値を調べたい場合（例えば、各遺伝子の発現が表現型値にどの程度関わっているか調べたい場合）、スケーリングをせずに解析すると、誤った結果を得ることになります。

### 2-4. モデルの評価

　モデルの評価でも、スケーリング後のトレーニングデータ・テストデータを使います。

In [ ]:
# モデルの評価: 決定係数R2
r2_train_ss = lr.score(x_train_ss, y_train)  # トレーニングデータ
r2_test_ss = lr.score(x_test_ss, y_test)    # テストデータ
print("training: ", r2_train_ss)
print("test: "    , r2_test_ss)

### モデル比較

　ここでは、モデル間の比較で重要な考え方を紹介します。

```
精度が高いモデルが良い
複雑なモデルより、単純なモデルが良い
```

　機械学習では、新しいデータに対して上手く予測できるモデル、すなわち「精度が高い」モデルが好まれます（実習で使った決定係数$R^2$はモデルの精度を測る指標です）。また、「単純なモデル」は、理解しやすいので好まれます。

　一般的に、モデルの精度を高くしようとすると、複雑になる傾向にあります。しかし、単純さを優先すると、精度が低いモデルになります。

　精度と単純さのバランスを評価する指標として、次のような指標があります。これらは、モデル間の比較に使われます。詳しく知りたい方は、統計モデリングの本を参照してください。
- AIC(赤池情報量基準): 予測能力が最良のモデルを良いとする指標
- BIC(ベイズ情報量規準): 真のモデルである確率が最も大きいモデルを良いとする指標


「統計モデリング」の勉強におすすめの本: 
- [データ解析のための統計モデリング入門](https://www.iwanami.co.jp/book/b257893.html)
- [Bayesian Analysis with Python](https://www.packtpub.com/big-data-and-business-intelligence/bayesian-analysis-python-second-edition)（原書）/ [Pythonによるベイズ統計モデリング](https://www.kyoritsu-pub.co.jp/bookdetail/9784320113374)（日本語訳）

### 2-5. 予測

　まだ改善の余地はありそうですが、1変数のときよりも良いモデルが得られました。新しい入力データをこの予測モデルに入れて、その予測値を調べてみましょう。

　ここでは、新しく3サンプルの遺伝子発現データが得られたとして、その表現型値を予測します。

$$
(gene_{7}, gene_{11}, gene_{28}) = (x_1, x_2, x_3) = (10.0, 9.0, 12.0), (8.0, 10.5, 13.0), (9.1, 12.3, 8.9)
$$

予測の手順:  
1. 新しいデータのスケーリングをします。トレーニングデータと同じ基準で、スケーリングする必要があります（テストデータのスケーリングと同じ操作です）。
```python
スケーリング名.transform(新しいデータ)
```

1. `predict`で新しいデータの予測値を得ます。
```python
モデル名.predict(スケーリング後のデータ)
```

In [ ]:
# 新しいデータ
x_new = np.array([(10.0, 9.0, 12.0), (8.0, 10.5, 13.0), (9.1, 12.3, 8.9)])

# スケーリング（標準化 standardization）
x_new_ss = ss.transform(x_new)

# 予測
lr.predict(x_new_ss)

---

## まとめ

　使用する説明変数の数を増やして、予測モデルの改善をおこないました。「機械学習 - 線形回帰（1）-」のテキストよりは良いスコア（決定係数）が得られました。

　このテキストではおこないませんでしたが、使用する説明変数を他のものに変えてみると、さらに良い予測モデルを構築できるかもしれません。興味があれば、さらに良い予測モデルの構築をおこなってみてください。

　次回は、目的関数（コスト関数）の最小値を探すアルゴリズム **勾配法 Gradient method** を学びます。勾配法は、機械学習では非常に重要なアルゴリズムです。高度な機械学習モデル、ニューラルネットワーク（Neural network）などにも使われています。

### おまけ: 遺伝子50個を使った予測モデル（悪い予測モデル）


　50個の遺伝子すべてのデータを使っても、良い予測モデルはできません。トレーニングデータの決定係数は高くなりますが、テストデータの決定係数は非常に低い値になります（**過学習**と言われる状態です）。良いモデルを作るためには、使用する遺伝子を選ぶ必要があります。

In [ ]:
# 悪い予測モデル: 遺伝子50個を使った線形回帰モデル
# 使用する変数
import numpy as np
x = np.array(df.loc[:,"gene_1":]) # 説明変数50個
y = np.array(df["phenotype"])   # 目的変数

# データ分割: 100サンプルを「トレーニングデータ70%、テストデータ30%」に分ける
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

# 標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train_ss = ss.fit_transform(x_train)
x_test_ss = ss.transform(x_test)

# モデル選択＆学習
from sklearn.linear_model import LinearRegression
lr50 = LinearRegression()
lr50.fit(x_train_ss, y_train)

# 係数b、誤差e
b = lr50.coef_
e = lr50.intercept_
# print("Coefficient=", b)
# print("Intercept="  , e)

# モデルの評価: 決定係数R2
r2_train_ss = lr50.score(x_train_ss, y_train)  # トレーニングデータ
r2_test_ss = lr50.score(x_test_ss, y_test)    # テストデータ
print("training: ", r2_train_ss)
print("test: "    , r2_test_ss)